how to run things

A simple(r) version of mimic.py

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# data related
import pandas as pd
from src.data.data_loader import MIMIC3DataModule
import json
from sklearn.model_selection import KFold
import numpy as np

# use lightning framework
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

# import models
from src.models.models import *
from src.models.output import *

# other
import os

# plotting
import matplotlib.pyplot as plt

In [3]:
seed_everything(1)

Global seed set to 1


1

### Data

In [4]:
df = pd.read_csv('../data/mimic.csv')
df.sort_values(by=['stay_id','timer'],inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,rn,subject_id,hadm_id,stay_id,timer,timer_next,timer_prev,glc,glc_next,glc_lab,...,tpn_rate,dextrose_fluid,delta_t,msk0,sum_msk0,n,gender_m,msk,timer_d,timer_next_d
0,0,12466550,23998182,30000153,2174-09-29 20:12:00+00:00,2174-09-30 13:00:00+00:00,2174-09-29 12:09:00+00:00,185.0,139.0,NaN,...,0.0,0.0,16.80,0.0,0.0,1,1.0,0.0,8.050000,24.850000
1,1,12466550,23998182,30000153,2174-09-30 13:00:00+00:00,2174-09-30 20:00:00+00:00,2174-09-29 20:12:00+00:00,139.0,137.0,NaN,...,0.0,0.0,7.00,0.0,0.0,2,1.0,0.0,24.850000,31.850000
2,3,13180007,27543152,30000213,2162-06-21 06:06:00+00:00,2162-06-21 07:00:00+00:00,2162-06-21 05:40:00+00:00,125.0,108.0,NaN,...,0.0,0.0,0.90,0.0,1.0,1,1.0,0.0,0.466667,1.366667
3,4,13180007,27543152,30000213,2162-06-21 07:00:00+00:00,2162-06-21 07:45:00+00:00,2162-06-21 06:06:00+00:00,108.0,97.0,NaN,...,0.0,0.0,0.75,0.0,1.0,2,1.0,0.0,1.366667,2.116667
4,5,13180007,27543152,30000213,2162-06-21 07:45:00+00:00,2162-06-21 08:15:00+00:00,2162-06-21 07:00:00+00:00,97.0,156.0,NaN,...,0.0,0.0,0.50,0.0,1.0,3,1.0,0.0,2.116667,2.616667


In [5]:
df.loc[df.stay_id == 30000213,:]

,rn,subject_id,hadm_id,stay_id,timer,timer_next,timer_prev,glc,glc_next,glc_lab,...,tpn_rate,dextrose_fluid,delta_t,msk0,sum_msk0,n,gender_m,msk,timer_d,timer_next_d
2,3,13180007,27543152,30000213,2162-06-21 06:06:00+00:00,2162-06-21 07:00:00+00:00,2162-06-21 05:40:00+00:00,125.0,108.0,NaN,...,0.0,0.0,0.900000,0.0,1.0,1,1.0,0.0,0.466667,1.366667
3,4,13180007,27543152,30000213,2162-06-21 07:00:00+00:00,2162-06-21 07:45:00+00:00,2162-06-21 06:06:00+00:00,108.0,97.0,NaN,...,0.0,0.0,0.750000,0.0,1.0,2,1.0,0.0,1.366667,2.116667
4,5,13180007,27543152,30000213,2162-06-21 07:45:00+00:00,2162-06-21 08:15:00+00:00,2162-06-21 07:00:00+00:00,97.0,156.0,NaN,...,0.0,0.0,0.500000,0.0,1.0,3,1.0,0.0,2.116667,2.616667
5,6,13180007,27543152,30000213,2162-06-21 08:15:00+00:00,2162-06-21 18:41:00+00:00,2162-06-21 07:45:00+00:00,156.0,164.0,NaN,...,0.0,0.0,10.433333,0.0,1.0,4,1.0,0.0,2.616667,13.050000
6,7,13180007,27543152,30000213,2162-06-21 18:41:00+00:00,2162-06-21 23:00:00+00:00,2162-06-21 08:15:00+00:00,164.0,225.0,NaN,...,0.0,0.0,4.316667,0.0,1.0,5,1.0,0.0,13.050000,17.366667
7,8,13180007,27543152,30000213,2162-06-21 23:00:00+00:00,2162-06-22 08:00:00+00:00,2162-06-21 18:41:00+00:00,225.0,296.0,NaN,...,0.0,0.0,9.000000,0.0,1.0,6,1.0,0.0,17.366667,26.366667
8,9,13180007,27543152,30000213,2162-06-22 08:00:00+00:00,2162-06-22 12:00:00+00:00,2162-06-21 23:00:00+00:00,296.0,239.0,NaN,...,0.0,0.0,4.000000,0.0,1.0,7,1.0,0.0,26.366667,30.366667


In [6]:
# subsample the data
icu_sample = np.random.choice(df.stay_id.unique(),200)
df = df.loc[df.stay_id.isin(icu_sample),:]
print(df.shape)

(4075, 54)


In [7]:
with open('../data/feature_sets.json', 'r') as f:
    feature_sets = json.load(f)
features = feature_sets['test_features']

In [8]:
features['timevarying'][0]

'glc'

In [9]:
dims = {'input_dim_t':len(features['timevarying']),
             'input_dim_0':len(features['static']),
             'input_dim_i':len(features['intervention']),
             'hidden_dim_t':8,
             'hidden_dim_0':None,
             'hidden_dim_i':4,
             'input_size_update':len(features['timevarying'])+len(features['static'])}

In [10]:
mimic = MIMIC3DataModule(features,df,df,batch_size=128,testing = False)
mimic.setup()

In [11]:
next(iter(mimic.train_dataloader()))

(tensor([[[ 0.8902],
          [ 0.7621],
          [ 0.7436],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],
 
         [[ 0.1643],
          [ 0.0623],
          [-0.2688],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],
 
         [[-0.1795],
          [ 0.0212],
          [ 0.0000],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],
 
         ...,
 
         [[-0.1054],
          [-0.1459],
          [ 0.0822],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],
 
         [[-0.3365],
          [-0.0975],
          [ 0.0000],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],
 
         [[ 0.0351],
          [-0.2054],
          [-0.0364],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]]]),
 tensor([[0., 0.],
         [0., 0.],
         [0., 0.],
         [1., 0.],
         [1., 1.],
         [0., 1.],
      

### Models

In [12]:
from src.models.base import BaseModel,BaseModelCT,BaseModelDT,BaseModelDecay
import torchctrnn as ct

from torch.nn.utils.parametrizations import spectral_norm

class Func(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.layers = nn.Sequential(
            spectral_norm(nn.Linear(input_dim, hidden_dim)),
            nn.Tanh(),
            spectral_norm(nn.Linear(hidden_dim, output_dim)),
        )

    def forward(self,hidden):
        output = self.layers(hidden)
        return output


class ctGRUModel(BaseModelCT):

    def __init__(self,dims,outputNN,preNN=nn.Identity(),NN0=nn.Identity(),learning_rate=1e-1,update_loss=0.1,merror=1e-2):
        func = Func(dims['hidden_dim_t'],50,dims['hidden_dim_t'])
        odenet = ct.NeuralODE(func,time_func='tanh',time_dependent=False,data_dependent=False,
                            solver='euler',solver_options={'step_size':1e-2})
        odernn = ct.ODEGRUCell(odenet,dims['input_size_update'],dims['hidden_dim_t'])
        outNN = outputNN(dims['hidden_dim_t'],g=g,ginv=ginv)
        super().__init__(odernn,outNN,preNN,NN0,dims,learning_rate,update_loss,merror)
        
        #self.save_hyperparameters({'net':'ctGRUModel'})

class dtGRUModel(BaseModelDT):

    def __init__(self,dims,outputNN,preNN=nn.Identity(),NN0=nn.Identity(),learning_rate=1e-2):
        odernn = nn.GRUCell(dims['input_size_update'],dims['hidden_dim_t'])
        outNN = outputNN(dims['hidden_dim_t'],g=g,ginv=ginv)
        super().__init__(odernn,outNN,preNN,NN0,dims,learning_rate)
        #self.save_hyperparameters({'net':'dtGRUModel'})

In [13]:
outputNN = GaussianOutputNNLL
model = ctGRUModel(dims,outputNN)

NeuralODE's forward method missing args: ['input', 't']. These are assumed not applicable


In [14]:
df.loc[df.glc.isnull(),:]

,rn,subject_id,hadm_id,stay_id,timer,timer_next,timer_prev,glc,glc_next,glc_lab,...,tpn_rate,dextrose_fluid,delta_t,msk0,sum_msk0,n,gender_m,msk,timer_d,timer_next_d


In [15]:
print('no change prediction error: {}'.format(np.sqrt(np.mean((df.glc - df.glc_next)**2))))

no change prediction error: 42.35775216538079


### Split

In [16]:
kf = KFold(n_splits=3)
ids_ = df.stay_id.unique()
splits = kf.split(ids_)

### Train

In [17]:
for i,(train_ids, test_ids) in enumerate(splits):

    df_test = df.loc[df[features['id']].isin(ids_[test_ids])].copy(deep=True)
    df_train = df.loc[df[features['id']].isin(ids_[train_ids])].copy(deep=True)

    mimic3 = MIMIC3DataModule(features,df_train,df_test,batch_size=32,testing = False)
    mimic3.setup()
    
    checkpoint_callback = ModelCheckpoint(monitor='val_loss',save_top_k=1)

    # train
    early_stopping = EarlyStopping(monitor="val_loss",mode="min",verbose=True,patience=10,min_delta=0.0)  # mostly defaults
    trainer = pl.Trainer(max_epochs=1)
    trainer.fit(model, mimic3)
    trainer.test(model,mimic3)

    break

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type               | Params
------------------------------------------------
0 | RNN      | ODEGRUCell         | 1.1 K 
1 | OutputNN | GaussianOutputNNLL | 82    
2 | preNN    | Identity           | 0     
3 | NN0      | Identity           | 0     
------------------------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model params size (MB)


/home/oisin/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1938: PossibleUserWarning: The number of training samples (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 3/3 [00:18<00:00,  6.19s/it]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      int_av_width          166.14633178710938
      int_coverage          0.9326599326599326
      int_med_width         141.61630249023438
        test_crps           0.17268215119838715
     test_ignorance        -0.28834763084139914
     test_int_score          247.2438507080078
        test_loss           0.2962607145309448
     test_loss_pred         0.28834760189056396
    test_loss_update        0.07913099974393845
        test_rmse            49.77627182006836
      test_var_pit          0.05740843540816496
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [19]:
predictions = trainer.predict(model,mimic3.test_dataloader())

Predicting DataLoader 0: 100%|██████████| 3/3 [00:15<00:00, -0.06it/s]   


In [24]:
predictions[0][:,0]

tensor([   139.,    141.,    142.,  ..., 125550., 125551., 125552.])

In [25]:
df.head()

,rn,subject_id,hadm_id,stay_id,timer,timer_next,timer_prev,glc,glc_next,glc_lab,...,tpn_rate,dextrose_fluid,delta_t,msk0,sum_msk0,n,gender_m,msk,timer_d,timer_next_d
135,139,12227720,29396704,30003749,2120-11-06 08:00:00+00:00,2120-11-06 09:07:00+00:00,2120-11-06 07:30:00+00:00,341.0,300.0,NaN,...,0.0,2.551020,1.116667,0.0,1.0,1,1.0,0.0,15.950000,17.066667
136,140,12227720,29396704,30003749,2120-11-06 09:07:00+00:00,2120-11-06 09:08:00+00:00,2120-11-06 08:00:00+00:00,300.0,NaN,NaN,...,0.0,4.734636,0.016667,0.0,1.0,2,1.0,1.0,17.066667,17.083333
137,141,12227720,29396704,30003749,2120-11-06 09:08:00+00:00,2120-11-06 10:07:00+00:00,2120-11-06 09:07:00+00:00,294.5,289.0,NaN,...,0.0,0.000000,0.983333,1.0,2.0,3,1.0,0.0,17.083333,18.066667
138,142,12227720,29396704,30003749,2120-11-06 10:07:00+00:00,2120-11-06 11:05:00+00:00,2120-11-06 09:08:00+00:00,289.0,233.0,NaN,...,0.0,0.956140,0.966667,0.0,2.0,4,1.0,0.0,18.066667,19.033333
139,143,12227720,29396704,30003749,2120-11-06 11:05:00+00:00,2120-11-06 12:00:00+00:00,2120-11-06 10:07:00+00:00,233.0,169.0,NaN,...,0.0,35.448876,0.916667,0.0,2.0,5,1.0,0.0,19.033333,19.950000
